In [1]:
using WriterVerifier

using Random
Random.seed!(42);

In [2]:
# Load data
println("Loading images...")

image_folder = "../data/words"

# Load images
writers = load_images(image_folder; max_per_writer=300);

println("Images loaded")

Loading images...
Images loaded


In [3]:
# Create pairs
pairs, labels = create_pairs(writers; positive=200, negative=200);

Creating positive pairs...
Creating negative pairs...
Created 398 pairs
Positive: 199
Negative: 199


In [4]:
# Create model
model = create_model()
println("Model created")


Model created


In [9]:
# Model training

trained_model, history = train_model!(
    model, pairs, labels;
    epochs=5, 
    batch_size=32,
    learning_rate=0.0001
)

final_acc = round(history["val_acc"][end] * 100, digits=1)
println("Final accuracy: $(final_acc)%")

Starting training...
Training data: 318
Validation data: 80

Epoch 1/5
Train loss: 0.6915
Val loss: 0.6932
Val accuracy: 48.8%

Epoch 2/5
Train loss: 0.6916
Val loss: 0.6923
Val accuracy: 52.5%

Epoch 3/5
Train loss: 0.6918
Val loss: 0.6933
Val accuracy: 43.8%

Epoch 4/5
Train loss: 0.6978
Val loss: 0.6918
Val accuracy: 53.8%

Epoch 5/5
Train loss: 0.6986
Val loss: 0.692
Val accuracy: 50.0%

Training completed!
Final accuracy: 50.0%


In [6]:
# Testing on real examples

# Select random test examples
idx = randperm(length(pairs))[1:min(6, length(pairs))]
test_pairs = pairs[idx]
test_labels = labels[idx]

for i in 1:length(test_pairs)
    path1, path2 = test_pairs[i]
    true_label = test_labels[i]
    
    # Extract writer names
    writer1 = split(basename(path1), "-")[1]
    writer2 = split(basename(path2), "-")[1]
    
    # Test similarity
    similarity = test_similarity(trained_model, path1, path2)
    
    # Interpret results
    prediction = similarity > 0.5 ? "SAME" : "DIFFERENT"
    truth = true_label == 1 ? "SAME" : "DIFFERENT"
    correct = (similarity > 0.5) == (true_label == 1) ? "✓" : "X"
    
    println("$writer1 and $writer2")
    println("Similarity: $(round(similarity, digits=3))")
    println("Prediction: $prediction | Truth: $truth $correct")
end


a01 and a01
Similarity: 0.506
Prediction: SAME | Truth: SAME ✓
a02 and a02
Similarity: 0.498
Prediction: DIFFERENT | Truth: SAME X
a05 and a05
Similarity: 0.497
Prediction: DIFFERENT | Truth: SAME X
a04 and a05
Similarity: 0.499
Prediction: DIFFERENT | Truth: DIFFERENT ✓
a02 and a01
Similarity: 0.5
Prediction: DIFFERENT | Truth: DIFFERENT ✓
a01 and a05
Similarity: 0.5
Prediction: DIFFERENT | Truth: DIFFERENT ✓


In [7]:
# Model saving
try
    if !isdir("../models")
        mkdir("../models")
    end
    
    save_model(trained_model, "../models/model.jld2")
    
catch e
    println("Save error: $e")
end

Model saved: ../models/model.jld2
